# Description: 
-This program uses a TensorFlow-trained neural network to perform object detection.

-It loads the classifier and uses it to perform object detection on an image.

-It draws boxes, scores, and labels around the objects of interest in the image.

# Import important packages and define important paths

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import imutils

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Number of classes the object detector can identify
NUM_CLASSES = 5

# Load the label map.
Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [2]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
print(label_map)

categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
print(categories)

category_index = label_map_util.create_category_index(categories)
print(category_index)


item {
  name: "Knife"
  id: 1
}
item {
  name: "Gun"
  id: 2
}
item {
  name: "Wrench"
  id: 3
}
item {
  name: "Pliers"
  id: 4
}
item {
  name: "Scissors"
  id: 5
}

[{'id': 1, 'name': 'Knife'}, {'id': 2, 'name': 'Gun'}, {'id': 3, 'name': 'Wrench'}, {'id': 4, 'name': 'Pliers'}, {'id': 5, 'name': 'Scissors'}]
{1: {'id': 1, 'name': 'Knife'}, 2: {'id': 2, 'name': 'Gun'}, 3: {'id': 3, 'name': 'Wrench'}, 4: {'id': 4, 'name': 'Pliers'}, 5: {'id': 5, 'name': 'Scissors'}}


# Load the Tensorflow model into memory.

In [3]:
# initialize the model
detection_graph = tf.Graph()

with detection_graph.as_default():
    # initialize the graph definition
    od_graph_def = tf.GraphDef()
    
    # load the graph from disk
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as f:
        serialized_graph = f.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors for the object detection classifier

In [4]:
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes

# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

# The score is shown on the result image, together with the class label.
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Let's start testing
we will make a for loop to test a complete folder 
we will load image using OpenCV and expand image dimensions to have shape: [1, None, None, 3]
i.e. a single-column array, where each item in the column has the pixel RGB value

In [6]:
from tqdm import tqdm
path_to_images_folder = "images for testing"
for i in tqdm(os.listdir(path_to_images_folder),total=len(os.listdir(path_to_images_folder))):
#     print(i)
    n=(i.split(".")[0])
    image = cv2.imread(os.path.join(path_to_images_folder,i))
    before=image.copy()
    
#     (H, W) = image.shape[:2]
#     # check to see if we should resize along the width
#     if W > H and W > 1000:
#         image = imutils.resize(image, width=1000)
#     # otherwise, check to see if we should resize along the
#     # height
#     elif H > W and H > 1000:
#         image = imutils.resize(image, height=1000)
        
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_expanded = np.expand_dims(image_rgb, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})

#     # Draw the results of the detection
#     COLORS = np.random.uniform(0, 255, size=(NUM_CLASSES , 3))
#     boxes = np.squeeze(boxes)
#     scores = np.squeeze(scores)
#     labels = np.squeeze(classes)
    
# #     print(classes)

#     for (box, score, label) in (zip(boxes, scores, classes[0])):
# #         print(box)
#         if score < 0.6:
#             continue

#         (startY, startX, endY, endX) = box
#         (H, W) = image.shape[:2]

#         startX = int(startX * W)
#         startY = int(startY * H)
#         endX = int(endX * W)
#         endY = int(endY * H)

#         # draw the prediction on the output image
#         label = category_index[label]
#         idx = int(label["id"]) - 1
#         label = "{}: {:.2f}".format(label["name"], score)
#         cv2.rectangle(image, (startX, startY), (endX, endY),COLORS[idx], 5)
#         y = startY - 10 if startY - 10 > 10 else startY + 10
#         cv2.putText(image, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLORS[idx], 2)
    
    
    
    # Draw the results of the detection
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.6)
    
#     print(boxes.shape)
    # All the results have been drawn on image. Now display the image.
    name=n+"-after.jpg"
    cv2.imwrite(name,image)
#     cv2.imshow("images", np.hstack([before, image]))
#     cv2.waitKey(0)
cv2.destroyAllWindows()

100%|██████████| 20/20 [00:03<00:00,  6.24it/s]
